In [1]:
!pip install rioxarray rasterio pystac_client planetary_computer odc.stac sentinelsat hvplot zarr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.4/255.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.1/155.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.8/206.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 3.1 MB/s eta 0:00:00


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import warnings 
warnings.filterwarnings('ignore')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio
from tqdm import tqdm
import pystac_client 
import planetary_computer
from odc.stac import stac_load

plt.rcParams['figure.figsize'] = (10,8)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# Orenburg (RU) – single Sentinel-2 tile footprint
# lower_left  = (51.10, 55.00)   # (lat, lon)
# upper_right = (51.50, 56.00)

# # bounding box for fergana (uzbekistan)
# lower_left  = (40.15, 70.75)        # (lat, lon)
# upper_right = (40.55, 71.75)
# 2000 m buffer: [70.98653092 39.99127677 73.00948379 42.01743359]
# bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0])
# Fergana  chip: [70.93608325 39.97031454 73.09979272 42.02397519]
# Orenburg chip: [53.86041435 50.94060491 56.04516069 53.02573486]

bounds_fergana = (70.98653092, 39.99127677, 73.00948379, 42.01743359 )
bounds_orenburg = (53.97714297, 50.9672661,  56.01383563, 53.0001858 )
time_window = "2021-07-01/2021-07-31"
time_window_fergana = "2020-06-01/2020-08-31"

In [4]:
stac = pystac_client.Client.open('https://planetarycomputer.microsoft.com/api/stac/v1')
search_fergana = stac.search(
    bbox=bounds_fergana,
    datetime=time_window_fergana,
    collections=['sentinel-2-l2a'],
    query={'eo:cloud_cover': {'lt': 5}},
)
search_orenburg = stac.search(
    bbox=bounds_orenburg,
    datetime=time_window,
    collections=['sentinel-2-l2a'],
    query={'eo:cloud_cover': {'lt': 5}},
)

In [5]:
items_fergana = list(search_fergana.get_items())
print(f'Number of Sentinel-2 scenes found :{len(items_fergana)}')
signed_items_fergana = [planetary_computer.sign(item) for item in items_fergana]

print('==============')

items_orenburg = list(search_orenburg.get_items())
print(f'Number of Sentinel-2 scenes found :{len(items_orenburg)}')
signed_items_orenburg = [planetary_computer.sign(item) for item in items_orenburg]

Number of Sentinel-2 scenes found :74
Number of Sentinel-2 scenes found :65


In [6]:
bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B11', 'B12', 'SCL', 'WVP', 'AOT']
resolution = 50/111320.0

In [7]:
data_fergana = stac_load(
    signed_items_fergana,
    bands=bands,
    crs='EPSG:4326',
    resolution=resolution,
    chunks={'x':2048, 'y':2048},
    dtype='uint16',
    patch_url = planetary_computer.sign,
    bbox=bounds_fergana,
)
# data_fergana = data_fergana.persist()
# print('done')
data_orenburg = stac_load(
    signed_items_orenburg,
    bands=bands,
    crs='EPSG:4326',
    resolution=resolution,
    chunks={'x':2048, 'y':2048},
    dtype='uint16',
    patch_url = planetary_computer.sign,
    bbox=bounds_orenburg,
)
# data_orenburg = data_orenburg.persist()

# per day

In [8]:
# unique_dates_fergana = data_fergana.time.dt.date.drop_duplicates("time")
# print("Unique dates:", unique_dates_fergana.size)

# first_per_day_fergana = data_fergana.groupby("time.date").first()
# data_fergana = first_per_day_fergana.rename(date="time") 

# print('===========')

# unique_dates_orenburg = data_orenburg.time.dt.date.drop_duplicates("time")
# print("Unique dates:", unique_dates_orenburg.size)

# first_per_day_orenburg = data_orenburg.groupby("time.date").first()
# data_orenburg = first_per_day_orenburg.rename(date="time") 

# over time

In [9]:
import numpy as np
import xarray as xr

# -------------------------
# Fergana
# -------------------------
unique_dates_fergana = data_fergana.time.dt.date.drop_duplicates("time")
print("Unique dates (Fergana):", unique_dates_fergana.size)

# group by date and mosaic tiles for each day
mosaics_fergana = []
for date, ds in data_fergana.groupby("time.date"):
    # take pixelwise max so valid pixels overwrite black nodata (0)
    mosaic = ds.max("time")
    # assign correct date back
    mosaic = mosaic.assign_coords(time=np.datetime64(date))
    mosaics_fergana.append(mosaic)

# combine into one dataset
data_fergana = xr.concat(mosaics_fergana, dim="time")

# -------------------------
# Orenburg
# -------------------------
unique_dates_orenburg = data_orenburg.time.dt.date.drop_duplicates("time")
print("Unique dates (Orenburg):", unique_dates_orenburg.size)

mosaics_orenburg = []
for date, ds in data_orenburg.groupby("time.date"):
    mosaic = ds.max("time")
    mosaic = mosaic.assign_coords(time=np.datetime64(date))
    mosaics_orenburg.append(mosaic)

data_orenburg = xr.concat(mosaics_orenburg, dim="time")

Unique dates (Fergana): 18
Unique dates (Orenburg): 10


In [10]:
# import hvplot.xarray  # noqa
# import panel as pn

# # Downsample for speed
# def downsample_for_view(da, factor=8):
#     return da.coarsen(longitude=factor, latitude=factor, boundary="pad").mean()

# # Convert to RGB-friendly structure
# def prep_rgb(ds):
#     return downsample_for_view(ds[["B04","B03","B02"]]).to_array("band")

# rgb_f = prep_rgb(data_fergana)
# rgb_o = prep_rgb(data_orenburg)

# # hvplot wants: r=, g=, b= mappings
# def plot_rgb(da, title):
#     return da.hvplot.rgb(
#         x="longitude", y="latitude",
#         r="B04", g="B03", b="B02",
#         clim=(0, 2500),  # scale like before
#         frame_width=400, title=title,
#         widget_type="scrubber"  # makes it animatable
#     )

# plot_f = plot_rgb(data_fergana, "Fergana")
# plot_o = plot_rgb(data_orenburg, "Orenburg")

# pn.Row(plot_f, plot_o).servable()


In [11]:
# import matplotlib.pyplot as plt
# from matplotlib.animation import FuncAnimation
# import numpy as np

# # -----------------------------------------------------------
# # 1.  One-time down-sample for speed
# # -----------------------------------------------------------
# def downsample_for_view(da, factor=8):
#     return da.coarsen(longitude=factor, latitude=factor, boundary="pad").mean()

# rgb_f = downsample_for_view(
#     data_fergana[["B04","B03","B02"]].to_array("band")
# ).transpose("time", "latitude", "longitude", "band")

# rgb_o = downsample_for_view(
#     data_orenburg[["B04","B03","B02"]].to_array("band")
# ).transpose("time", "latitude", "longitude", "band")

# # -----------------------------------------------------------
# # 2.  Helper to build an animation for one cube
# # -----------------------------------------------------------
# def quick_anim(rgb, title):
#     n = rgb.sizes["time"]
#     fig, ax = plt.subplots(figsize=(5, 5))
#     ax.set_title(f"{title} — day 0")
#     im = ax.imshow(np.clip(rgb[0].values / 2500, 0, 1), origin="upper")
#     ax.axis("off")

#     def update(i):
#         im.set_data(np.clip(rgb[i].values / 2500, 0, 1))
#         ax.set_title(f"{title} — {str(rgb.time[i].values)[:10]}")
#         return im,

#     anim = FuncAnimation(fig, update, frames=n, interval=200, blit=True)
#     plt.close(fig)          # keeps notebook clean
#     return anim

# # -----------------------------------------------------------
# # 3.  Build two animations and show them inline
# # -----------------------------------------------------------
# anim_f = quick_anim(rgb_f, "Fergana")
# # anim_o = quick_anim(rgb_o, "Orenburg")

# # In a Jupyter cell:
# from IPython.display import HTML
# display(HTML(anim_f.to_jshtml()))
# # display(HTML(anim_o.to_jshtml()))

In [12]:
# # valid_ratio = (data[["B04","B03","B02"]] > 0).all(dim="time").mean(dim=("longitude","latitude"))
# # data = data.where(valid_ratio > 0.05, drop=True)

# # valid_mask = (data["B04"] > 0)  # True where valid, False where black/missing

# # # Step 2: Compute % valid pixels per time slice
# # valid_ratio = valid_mask.mean(dim=("longitude", "latitude"))

# # # Step 3: Keep only scenes with ≥95% valid pixels
# # data = data.where(valid_ratio >= 0.95, drop=True)

# # rgb_fergana = data_fergana[["B04"]].to_array(dim="band")
# # scene_ok_fergana = (rgb_fergana > 0).all("band").mean(("longitude", "latitude"))
# # mask_fergana = scene_ok_fergana.compute() >= 0.8   
# # rgb_orenburg = data_orenburg[["B04"]].to_array(dim="band")
# # scene_ok_orenburg = (rgb_orenburg > 0).all("band").mean(("longitude", "latitude"))
# # mask_orenburg = scene_ok_orenburg.compute() >= 0.8   
# # data_fergana = data_fergana.isel(time=mask_fergana)
# # data_orenburg = data_orenburg.isel(time=mask_orenburg)
# # print(f"Remaining scenes: {data_fergana.time.size}")
# # print(f"Remaining scenes: {data_orenburg.time.size}")

# # import xarray as xr  # Assuming already imported

# # Define coarsen factor (e.g., 10 reduces spatial data by ~100x for mask calc)
# coarsen_factor = 10

# # For Fergana: Simplify and coarsen before mean
# valid_frac_fergana = (data_fergana["B04"].coarsen(longitude=coarsen_factor, latitude=coarsen_factor, boundary='pad').mean() > 0).mean(("longitude", "latitude"))
# mask_fergana = valid_frac_fergana >= 0.8  # This is lazy
# mask_fergana = mask_fergana.compute()  # Now compute (much smaller data)

# # For Orenburg
# valid_frac_orenburg = (data_orenburg["B04"].coarsen(longitude=coarsen_factor, latitude=coarsen_factor, boundary='pad').mean() > 0).mean(("longitude", "latitude"))
# mask_orenburg = valid_frac_orenburg >= 0.8
# mask_orenburg = mask_orenburg.compute()

# # Slice the data (original resolution preserved)
# data_fergana = data_fergana.isel(time=mask_fergana)
# data_orenburg = data_orenburg.isel(time=mask_orenburg)

# print(f"Remaining scenes Fergana: {data_fergana.time.size}")
# print(f"Remaining scenes Orenburg: {data_orenburg.time.size}")

In [13]:
# n_scenes = len(data["time"])
# print("Scenes left after filter:", n_scenes)

In [14]:
# data = data.persist()
# print(data)
# display(data_orenburg)

In [15]:
# for band in bands:
#     data.isel(time=0)[band].plot.imshow(cmap='viridis', robust=True)
#     plt.title(f"{band} Band")
#     plt.axis('off')
#     plt.show()

In [16]:
# plot_data_fergana = data_fergana[["B04","B03","B02"]].to_array()
# plot_data_orenburg = data_orenburg[["B04","B03","B02"]].to_array()

# from matplotlib.animation import FuncAnimation
# import xarray as xr  # Assuming this is already imported

# # Define downsample factor (e.g., 4 reduces data by ~16x; increase for more speed)
# downsample_factor = 4

# # Function to downsample a single time step (fixed dims)
# def downsample_frame(data, time_idx):
#     return data.isel(time=time_idx).coarsen(
#         longitude=downsample_factor, latitude=downsample_factor, boundary='pad'
#     ).mean()

# def precompute_frames(plot_data, downsample_factor):
#     """Return a list of coarse arrays already transposed to (lat, lon, band)."""
#     frames = []
#     for t in range(plot_data.sizes['time']):
#         coarse = (plot_data.isel(time=t)
#                   .coarsen(longitude=downsample_factor,
#                            latitude=downsample_factor,
#                            boundary='pad')
#                   .mean()
#                   .transpose('latitude', 'longitude', 'variable')
#                   .values)
#         frames.append(coarse)
#     return frames

# def create_animation_fast(frames, title_prefix):
#     num_frames = len(frames)
#     fig, ax = plt.subplots(figsize=(6, 6))
#     ax.axis('off')
#     im = ax.imshow(frames[0], vmin=0, vmax=2500)
#     title = ax.set_title(f"{title_prefix} - Time 0")

#     def update(frame):
#         im.set_data(frames[frame])
#         title.set_text(f"{title_prefix} - Time {frame}")
#         return im,

#     anim = FuncAnimation(fig, update, frames=num_frames,
#                          interval=500, blit=True)
#     plt.close(fig)
#     return anim

# # ---- run once ----
# frames_fergana = precompute_frames(plot_data_fergana, downsample_factor)
# frames_orenburg = precompute_frames(plot_data_orenburg, downsample_factor)

# # ---- build animations ----
# anim_fergana  = create_animation_fast(frames_fergana,  "Fergana")
# anim_orenburg = create_animation_fast(frames_orenburg, "Orenburg")

# # In Jupyter, display with:
# from IPython.display import HTML
# HTML(anim_fergana.to_jshtml())
# HTML(anim_orenburg.to_jshtml())

# # Alternatively, save to file (e.g., for non-Jupyter use):
# # anim_fergana.save('fergana_animation.gif', writer='pillow', fps=2)
# # anim_orenburg.save('orenburg_animation.gif', writer='pillow', fps=2)
# # plot_data_fergana.plot.imshow(col='time', col_wrap=4, robust=True, vmin=0, vmax=2500)
# # plot_data_orenburg.plot.imshow(col='time', col_wrap=4, robust=True, vmin=0, vmax=2500)
# # plt.show()
# # rgb = data[["B04","B03","B02"]].to_array(dim="band")
# # rgb = rgb.transpose("time", "latitude", "longitude", "band")
# # anim = rgb.plot.imshow(col="time", robust=True, vmin=0, vmax=2500).animate()
# # from matplotlib import rc
# # rc('animation', html='jshtml')
# # anim

In [17]:

# downsample_factor = 4

# # For Fergana
# fig, ax = plt.subplots(figsize=(6, 6))
# downsampled_fergana = plot_data_fergana.isel(time=0).coarsen(
#     x=downsample_factor, y=downsample_factor, boundary='pad' 
# ).mean() 
# downsampled_fergana.plot.imshow(robust=True, ax=ax, vmin=0, vmax=2500)
# ax.set_title("demo date")
# ax.axis('off')
# plt.show()

# # For Orenburg
# fig, ax = plt.subplots(figsize=(6, 6))
# downsampled_orenburg = plot_data_orenburg.isel(time=8).coarsen(
#     x=downsample_factor, y=downsample_factor, boundary='pad'
# ).mean()
# downsampled_orenburg.plot.imshow(robust=True, ax=ax, vmin=0, vmax=2500)
# ax.set_title("demo date")
# ax.axis('off')
# plt.show()

In [18]:
# median = data.median(dim='time').compute()

In [19]:
# fig, ax = plt.subplots(figsize=(6,6))
# median[["B04", "B03", "B02"]].to_array().plot.imshow(robust=True, ax=ax, vmin=0, vmax=2500)
# ax.set_title("RGB Median Composite")
# ax.axis('off')
# plt.show()

In [20]:
# ndvi_median = (median.B08 - median.B04) / (median.B08 + median.B04)
# # nvdi = nvdi.persist()
# ndbi_median = (median.B11 - median.B08) / (median.B11 + median.B08)
# ndwi_median = (median.B03 - median.B08) / (median.B03 + median.B08)

**Enhanced Vegetation Index(EVI)**

**Soil-Adjusted Vegetation Index(SAVI)**

**Modified Soil_Adjusted Vegetation Inde(MSAVI)**

**Green Normalized Difference Vegetation Index (GNDVI)**

**Index-Based Built-Up Index(IBI)**

**Urban Index(UI)**

**Normalized Difference Bare Soil Index(NDSI)**

**Normalized Difference Moisture 
Index(NDMI)**

**Normalized Difference Snow Index(NDSI)**

**Brightness Index(BI)**



# Weighted Mosaicing

In [21]:
import numpy as np
import rasterio
import xarray as xr
import rioxarray as rxr
import gc
import psutil
from datetime import datetime
from tqdm import tqdm
import os
import shutil
import zarr  # Ensure zarr is imported

def mem():
    """Current RAM in GB"""
    return psutil.Process().memory_info().rss / 1024**3

def _valid_mask(da, scl=None):
    """
    True = usable pixel (not NaN, not black, not cloud).
    da: DataArray for one band
    scl: DataArray cloud mask (optional)
    """
    not_nan   = ~xr.ufuncs.isnan(da)
    not_black = (da != 0)          # adjust if your black value differs
    mask      = not_nan & not_black
    if scl is not None:
        cloud = (scl == 8) | (scl == 9) | (scl == 10) | (scl == 11)
        mask &= ~cloud
    return mask.astype("float32")   # 1=valid, 0=invalid

def build_weighted_mosaic_ultra(data: xr.Dataset, prefix: str):
    """
    data: xarray Dataset with dims (time, y, x) and bands + optional 'SCL'
    prefix: 'fergana' or 'orenburg'
    """
    print("=" * 60)
    print(f"[{prefix}] START   {mem():.1f} GB")
    bands = [b for b in data.data_vars if b != "SCL"]
    if not bands:
        raise ValueError("No bands")

    # Initialize for weighted mean
    running_sum = {}
    weight_sum = None  # Will initialize with first valid

    # For median, prepare temp zarr
    temp_dir = f"{prefix}_temp_zarr"
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    first_append = {b: True for b in bands}

    # Process each date incrementally
    num_valid_dates = 0
    for t in tqdm(data.time.values, desc=f"[{prefix}] Processing dates"):
        try:
            # Load this time slice
            da_t = data.sel(time=t).load()
            scl_t = da_t.get("SCL")
            valid_t = _valid_mask(da_t[bands[0]], scl_t)

            # Skip if all invalid
            if valid_t.sum() == 0:
                print(f"Skipping fully invalid date {t}")
                continue

            num_valid_dates += 1

            # Accumulate for mean
            if weight_sum is None:
                weight_sum = xr.zeros_like(valid_t)
            weight_sum += valid_t

            for b in bands:
                band_t = da_t[b]
                weighted = band_t * valid_t
                if b not in running_sum:
                    running_sum[b] = xr.zeros_like(weighted)
                running_sum[b] += weighted

            # Append to zarr for median
            for b in bands:
                valid_band = da_t[b].where(valid_t == 1, np.nan)
                if first_append[b]:
                    valid_band.expand_dims(time=[t]).to_zarr(temp_dir, mode='w', group=b, consolidated=True)
                    first_append[b] = False
                else:
                    valid_band.expand_dims(time=[t]).to_zarr(temp_dir, append_dim='time', group=b, consolidated=True)

            # Cleanup
            del da_t, valid_t, scl_t
            gc.collect()

        except Exception as e:
            print(f"Skipping bad date {t}: {str(e)}")
            continue

    if num_valid_dates == 0:
        raise ValueError("No valid dates")

    print(f"[{prefix}] Valid dates processed: {num_valid_dates} / {len(data.time)}")

    # Compute weighted mean
    w_mean = {}
    for b in tqdm(bands, desc=f"[{prefix}] Computing mean"):
        w_mean[b] = xr.where(weight_sum > 0, running_sum[b] / weight_sum, np.nan)
        del running_sum[b]  # Free mem
        gc.collect()

    mean_ds = xr.Dataset(w_mean)

    # Compute weighted median from zarr
    w_median = {}
    for b in tqdm(bands, desc=f"[{prefix}] Computing median"):
        z_da = xr.open_zarr(temp_dir, group=b, chunks='auto')[b]
        w_median[b] = z_da.median(dim="time", skipna=True).compute()
        del z_da
        gc.collect()

    median_ds = xr.Dataset(w_median)

    # Cleanup temp zarr
    shutil.rmtree(temp_dir)

    # Save
    mean_file   = f"{prefix}_weighted_mean.tif"
    median_file = f"{prefix}_weighted_median.tif"
    compress = dict(compress="lzw", tiled=True)

    mean_ds.rio.to_raster(mean_file, **compress)
    median_ds.rio.to_raster(median_file, **compress)

    print(f"[{prefix}] SAVED   {mean_file}")
    print(f"[{prefix}] SAVED   {median_file}")
    print(f"[{prefix}] END     {mem():.1f} GB")
    print("=" * 60)

    return mean_ds, median_ds

# Ultra-safe usage - process one location at a time with maximum caution
print("=" * 60)
print("ULTRA-SAFE PROCESSING FOR KAGGLE")
print("=" * 60)

try:
    print("Processing Fergana...")
    fergana_mean, fergana_median = build_weighted_mosaic_ultra(data_fergana, "fergana")
    
    # Aggressive cleanup before next location
    del data_fergana, fergana_mean, fergana_median
    gc.collect()
    
    print("\n" + "=" * 60)
    print("Processing Orenburg...")
    orenburg_mean, orenburg_median = build_weighted_mosaic_ultra(data_orenburg, "orenburg")
    
    print("\n" + "=" * 60)
    print("ALL PROCESSING COMPLETE!")
    print("=" * 60)
    
except Exception as e:
    print(f"CRITICAL ERROR: {e}")
    print("Try reducing chunk sizes further or processing fewer dates at a time")

ULTRA-SAFE PROCESSING FOR KAGGLE
Processing Fergana...
[fergana] START   0.4 GB


[fergana] Processing dates:  11%|█         | 2/18 [01:18<10:16, 38.53s/it]

Skipping bad date 2020-06-15T00:00:00: 'Float64' object has no attribute 'value'


[fergana] Processing dates:  17%|█▋        | 3/18 [02:13<11:31, 46.10s/it]

Skipping bad date 2020-06-20T00:00:00: 'Float64' object has no attribute 'value'


...

Skipping bad date 2020-08-24T00:00:00: 'Float64' object has no attribute 'value'


[fergana] Processing dates: 100%|██████████| 18/18 [20:56<00:00, 69.81s/it]


Skipping bad date 2020-08-29T00:00:00: 'Float64' object has no attribute 'value'
[fergana] Valid dates processed: 18 / 18


[fergana] Computing median:   0%|          | 0/13 [00:00<?, ?it/s]

CRITICAL ERROR: 'Float64' object has no attribute 'value'
Try reducing chunk sizes further or processing fewer dates at a time


In [22]:
# # # Weighted Temporal Mosaicking
# # # Assign weights based on cloud-free pixels (cloud mask using SCL band)
# weights = ~((data_fergana["SCL"] == 8) | 
#             (data_fergana["SCL"] == 9) | 
#             (data_fergana["SCL"] == 10) | 
#             (data_fergana["SCL"] == 11))
# weights = weights.astype("float32")  # Convert to numeric mask
# print('weights done')
# # Build weighted mosaic
# weighted_mosaic = {}
# for band in data_fergana.data_vars.keys():
#     if band == "SCL":  # skip classification band
#         continue
#     num = (data_fergana[band] * weights).sum(dim="time")
#     denom = weights.sum(dim="time")
#     weighted_mosaic[band] = xr.where(denom > 0, num / denom, float("nan"))  # avoid div by zero

# # Convert to dataset
# weighted_mosaic_data = xr.Dataset(weighted_mosaic)
# print('weighted mosaic done')
# median_mosaic = {}
# for band in data_fergana.data_vars.keys():
#     if band == "SCL":
#         continue
#     valid = data_fergana[band].where(weights == 1)
#     median_mosaic[band] = valid.median(dim="time", skipna=True)

# median_mosaic_data = xr.Dataset(median_mosaic)
# print('median mosaic done')


# weighted_mosaic_tiff_path = "Sentinel2_WeightedMosaic_mean.tiff"
# weighted_mosaic_median_tiff_path = "Sentinel2_WeightedMosaic_median.tiff"
# # weighted_mosaic_data = weighted_mosaic_data.persist()
# # median_mosaic_data = median_mosaic_data.persist()
# weighted_mosaic_data.rio.to_raster(
#     "weighted_mean_mosaic_fergana.tif",
#     tiled=True,
#     windowed=True,
#     compress="lzw"
# )
# print('weighed mean mosaic saved')
# median_mosaic_data.rio.to_raster(
#     "weighted_median_mosaic_fergana.tif",
#     tiled=True,
#     windowed=True,
#     compress="lzw"
# )
# print('weighted median mosaic saved')
# print(f"Weighted mosaic GeoTIFF with all features saved at {weighted_mosaic_tiff_path}")


In [23]:
# weighted_mosaic_data = weighted_mosaic_data.compute()
# rgb = weighted_mosaic_data[["B04","B03","B02"]].to_array("band") / 3000
# rgb = rgb.clip(0, 1)   # use xarray's clip, not numpy’s

# rgb.plot.imshow(
#     col="band", col_wrap=3, robust=True, cmap="viridis"
# )

# # plt.imshow(np.transpose(rgb.values, (1,2,0)))
# plt.title("Mosaic (Median Composite)")
# plt.axis("off")
# plt.show()


In [24]:
# # after the cell above runs
# var_names = list(weighted_mosaic_data.data_vars)
# print("Variables in the single-scene dataset:")
# for v in var_names:
#     print(v)

In [25]:
# # 1. materialise the data
# rgb = weighted_mosaic_data[["B04","B03","B02"]].to_array(dim="band") * 0.0001
# rgb = rgb.transpose("latitude","longitude","band").compute()

# # 2. inspect actual value range
# print("min:", rgb.min().values, "max:", rgb.max().values)

# # 3. plot
# fig, ax = plt.subplots(figsize=(6,6))
# rgb.plot.imshow(robust=True, ax=ax, vmin=0, vmax=0.35)
# ax.set_title("RGB Composite (25 Jul)")
# ax.axis('off')
# plt.show()

In [26]:
# import rioxarray
# from rasterio.enums import Resampling
# import numpy as np
# import os
# import xarray as xr

# # def save_resampled_weighted_mosaics(
# #     ds,
# #     target_resolution=30,
# #     methods=['bilinear', 'cubic'],
# #     output_prefix='WeightedMosaic_30m'
# # ):
# #     """
# #     Resamples the weighted mosaic dataset to a specified resolution using multiple interpolation methods
# #     and saves each resampled dataset as a separate GeoTIFF file.
# #     Also prints per-band NaN statistics for diagnostic purposes.
    
# #     Parameters
# #     ----------
# #     ds : xarray.Dataset
# #         The weighted mosaic dataset.
# #     target_resolution : float, optional
# #         Desired output resolution in meters (default=30).
# #     methods : list of str, optional
# #         Interpolation methods to apply (default=['bilinear', 'cubic']).
# #     output_prefix : str, optional
# #         Prefix for output filenames (default='WeightedMosaic_30m').
# #     """
    
# #     resampling_methods = {
# #         'bilinear': Resampling.bilinear,
# #         'cubic': Resampling.cubic
# #     }
# #     target_crs = "EPSG:4326"  # UTM Zone 18N for NYC
    
# #     # ----------------------------------------------------------------
# #     # 1) Set CRS correctly
# #     # ----------------------------------------------------------------
# #     if not ds.rio.crs:
# #         ds = ds.rio.write_crs("EPSG:4326")
# #         print("CRS set to EPSG:4326.")
# #     else:
# #         print(f"Existing CRS: {ds.rio.crs}")
    
# #     # ----------------------------------------------------------------
# #     # 2) Rename dimensions if necessary
# #     # ----------------------------------------------------------------
# #     if 'latitude' in ds.dims and 'longitude' in ds.dims:
# #         ds = ds.rename({'latitude': 'y', 'longitude': 'x'})
# #         print("Renamed (latitude->y, longitude->x) for spatial dims.")
    
# #     # ----------------------------------------------------------------
# #     # 3) Resample each method
# #     # ----------------------------------------------------------------
# #     for method in methods:
# #         if method not in resampling_methods:
# #             print(f"Skipping invalid method: {method}")
# #             continue

# #         print(f"\n--- Processing {method.upper()} resampling ---")
        
# #         try:
# #             # Identify 2D spatial variables
# #             spatial_vars = [var for var in ds.data_vars if ds[var].ndim == 2]
# #             ds_filtered = ds[spatial_vars]
            
# #             processed_vars = {}
# #             for var_name in spatial_vars:
# #                 da = ds_filtered[var_name].astype(np.float32)
                
# #                 # Reproject to target CRS without setting nodata
# #                 da_reproj = da.rio.reproject(
# #                     dst_crs=target_crs,              # Correct keyword argument
# #                     resolution=target_resolution,
# #                     resampling=resampling_methods[method],
# #                     nodata=None                       # Do not set nodata
# #                 )
# #                 processed_vars[var_name] = da_reproj
            
# #             # Rebuild dataset
# #             ds_reproj = xr.Dataset(processed_vars)
            
# #             # Print NaN statistics
# #             print("NaN Stats (after resampling) for each variable:")
# #             for var_name in spatial_vars:
# #                 da_ = ds_reproj[var_name]
# #                 nan_count = da_.isnull().sum().compute().item()
# #                 total_count = da_.size
# #                 min_val = da_.min().compute().item()
# #                 max_val = da_.max().compute().item()
# #                 nan_percentage = (nan_count / total_count) * 100
# #                 print(f"  {var_name}: NaNs={nan_count}/{total_count} ({nan_percentage:.2f}%), Min={min_val:.4f}, Max={max_val:.4f}")
            
# #             # Define output path
# #             output_path = f"{output_prefix}_{method}.tiff"
# #             os.makedirs(os.path.dirname(output_prefix), exist_ok=True)
            
# #             # Save to GeoTIFF without nodata
# #             ds_reproj[spatial_vars].rio.to_raster(
# #                 output_path,
# #                 dtype="float32",
# #                 compress="LZW",
# #                 tiled=True,
# #                 nodata=None
# #             )
            
# #             print(f"✅ Saved resampled mosaic: {output_path}")
# #             print(f"Output dimensions: {ds_reproj.dims}")
        
# #         except Exception as e:
# #             print(f"❌ Error in {method} processing: {str(e)}")
# #             continue


# # # Usage with your dataset:
# # save_resampled_weighted_mosaics(weighted_mosaic_data, output_prefix='/kaggle/working/Sentinel2_Mosaic_bands_sampled')

In [27]:
# from rasterio.transform import from_bounds
# def resample_to_30m(ds, method="bilinear"):
#     """
#     Resamples Sentinel data from ~10m to ~30m (1/3 the resolution),
#     using bilinear or cubic interpolation, 
#     while preserving the 'latitude' and 'longitude' dimension names in the output.

#     Parameters:
#     - ds : xarray.Dataset
#         Sentinel dataset to resample (dims: 'time', 'latitude', 'longitude').
#     - method : str
#         Interpolation method ('bilinear' or 'cubic').

#     Returns:
#     - xarray.Dataset
#         Resampled dataset with lat/lon dimension names restored.
#     """
#     print(f"Resampling data to ~30m using {method} interpolation...")

#     # Define resampling methods
#     resampling_methods = {
#         "bilinear": Resampling.bilinear,
#         "cubic": Resampling.cubic
#     }
#     if method not in resampling_methods:
#         raise ValueError("Invalid method. Choose 'bilinear' or 'cubic'.")

#     # Store original lat/lon range
#     min_lat = float(ds.latitude.min().values)
#     max_lat = float(ds.latitude.max().values)
#     min_lon = float(ds.longitude.min().values)
#     max_lon = float(ds.longitude.max().values)

#     # Temporarily rename dims from (latitude, longitude) -> (y, x)
#     ds_renamed = ds.rename_dims({"latitude": "y", "longitude": "x"})
#     ds_renamed = ds_renamed.rename_vars({"latitude": "y", "longitude": "x"})

#     # Let rioxarray know these are spatial dims in EPSG:4326
#     ds_renamed = ds_renamed.rio.write_crs("EPSG:4326", inplace=False)
#     ds_renamed = ds_renamed.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)

#     # Determine new shape (1/3 factor → integer downsampling)
#     # e.g., if original ~10m, new ~30m => shape is 1/3 the original size
#     new_height = ds.sizes["latitude"] // 3
#     new_width  = ds.sizes["longitude"] // 3

#     # Reproject/resample with the new shape
#     resampled_ds = ds_renamed.rio.reproject(
#         ds_renamed.rio.crs,
#         resampling=resampling_methods[method],
#         shape=(new_height, new_width)
#     )

#     # Create new lat/lon arrays
#     new_lat = np.linspace(min_lat, max_lat, new_height)
#     new_lon = np.linspace(min_lon, max_lon, new_width)

#     # Assign these as coords on y/x
#     resampled_ds = resampled_ds.assign_coords(y=("y", new_lat), x=("x", new_lon))

#     # Rename dims/vars back to (latitude, longitude)
#     resampled_ds = resampled_ds.rename_dims({"y": "latitude", "x": "longitude"})
#     resampled_ds = resampled_ds.rename_vars({"y": "latitude", "x": "longitude"})

#     # Mark them again as the spatial dims
#     resampled_ds = resampled_ds.rio.set_spatial_dims(
#         x_dim="longitude", y_dim="latitude", inplace=False
#     )

#     print("✅ Resampling completed with coordinates restored.")
#     return resampled_ds

# def save_mosaic_to_tiff(ds, output_path, bounds):
#     """
#     Saves an xarray.Dataset (with dims "latitude" and "longitude") 
#     to a multi-band GeoTIFF, using a bounding box transform.

#     Parameters:
#     - ds : xarray.Dataset with .rio CRS = EPSG:4326
#     - output_path : str, path to save the GeoTIFF
#     - bounds : tuple (min_lon, min_lat, max_lon, max_lat)
#     """
#     os.makedirs(os.path.dirname(output_path), exist_ok=True)

#     # Identify spatial variables: 2D variables
#     spatial_vars = [var for var in ds.data_vars if ds[var].ndim == 2]

#     if not spatial_vars:
#         print("❌ No 2D spatial variables found to save.")
#         return

#     # Ensure all spatial_vars have the same shape
#     shapes = {ds[var].shape for var in spatial_vars}
#     if len(shapes) != 1:
#         print("❌ Not all spatial variables have the same shape.")
#         for var in spatial_vars:
#             print(f"{var}: shape={ds[var].shape}")
#         return

#     # Rasterio transform, using # of pixels from ds and bounding box from 'bounds'
#     width = ds.sizes["longitude"]
#     height = ds.sizes["latitude"]
#     transform = from_bounds(bounds[0], bounds[1], bounds[2], bounds[3], width, height)

#     # Stack spatial_vars into (count, height, width)
#     band_stack = np.stack([ds[var].values for var in spatial_vars])

#     print(f"Saving {len(spatial_vars)}-band GeoTIFF at: {output_path}")
#     with rasterio.open(
#         output_path,
#         "w",
#         driver="GTiff",
#         height=height,
#         width=width,
#         count=len(spatial_vars),
#         dtype="float32",
#         crs="EPSG:4326",
#         transform=transform,
#         compress="lzw"
#     ) as dst:
#         for i, var_name in enumerate(spatial_vars, start=1):
#             dst.write(band_stack[i - 1], i)
#             dst.set_band_description(i, var_name)

#     print(f"✅ Successfully saved: {output_path}")

#     # Print NaN statistics
#     print("NaN Stats for each band:")
#     for var_name in spatial_vars:
#         da = ds[var_name]
#         nan_count = np.isnan(da.values).sum()
#         total_count = da.size
#         nan_percentage = (nan_count / total_count) * 100
#         min_val = np.nanmin(da.values)
#         max_val = np.nanmax(da.values)
#         print(f"  {var_name}: NaNs={nan_count}/{total_count} ({nan_percentage:.2f}%), Min={min_val:.4f}, Max={max_val:.4f}")
# def verify_output_tiff(tiff_path):
#     """
#     Simple helper to check the existence and some metadata
#     of the output GeoTIFF file.
#     """
#     try:
#         tiff_path = os.path.abspath(tiff_path)
#         if not os.path.exists(tiff_path):
#             print(f"Error: File not found at {tiff_path}")
#             return False

#         print(f"\nVerifying file at: {tiff_path}")
#         with rasterio.open(tiff_path) as src:
#             print(f"CRS: {src.crs}")
#             print(f"Transform: {src.transform}")
#             print(f"Bounds: {src.bounds}")
#             file_mb = os.path.getsize(tiff_path)/(1024*1024)
#             print(f"File size: {file_mb:.2f} MB")

#         return True
#     except Exception as e:
#         print(f"❌ Error verifying TIFF: {e}")
#         return False
# resampled_30m_ds = resample_to_30m(weighted_mosaic_data, method="bilinear")
# save_mosaic_to_tiff(resampled_30m_ds, "/kaggle/working/sentinel_30m_billinear.tiff", bounds)

In [28]:
# resampled_30m_ds = resample_to_30m(weighted_mosaic_data, method="cubic")
# save_mosaic_to_tiff(resampled_30m_ds, "/kaggle/working/sentinel_30m_resampled_cubic.tiff", bounds)

In [29]:
# # Weighted Temporal Mosaicking
# # Assign weights based on cloud-free pixels (cloud mask using SCL band)
# # weights = ~((data["SCL"] == 8) | (data["SCL"] == 9) | (data["SCL"] == 10) | (data["SCL"] == 11))

# # # Calculate weighted mosaic
# # weighted_mosaic = {}
# # for band in data.data_vars.keys():
# #     # if band != "SCL":  # Skip the SCL layer
# #       weighted_mosaic[band] = (data[band] * weights).sum(dim="time") / weights.sum(dim="time")

# # Convert the weighted mosaic to an Xarray dataset
# scene = data.isel(time=8)          # or .sel(time='2023-07-25')

# # 2. feed it straight into "weighted_mosaic" dict
# weighted_mosaic = {band: scene[band] for band in scene.data_vars}

# # Convert the weighted mosaic to an Xarray dataset
# weighted_mosaic_data = xr.Dataset(weighted_mosaic)

# for var in weighted_mosaic_data.data_vars:
#     weighted_mosaic_data[var] = weighted_mosaic_data[var].astype("float32")

# # Add derived indices to the weighted mosaic
# # weighted_mosaic_data["NDVI"] = (weighted_mosaic_data.B08 - weighted_mosaic_data.B04) / (weighted_mosaic_data.B08 + weighted_mosaic_data.B04)
# # weighted_mosaic_data["NDBI"] = (weighted_mosaic_data.B11 - weighted_mosaic_data.B08) / (weighted_mosaic_data.B11 + weighted_mosaic_data.B08)
# # weighted_mosaic_data["NDWI"] = (weighted_mosaic_data.B03 - weighted_mosaic_data.B08) / (weighted_mosaic_data.B03 + weighted_mosaic_data.B08)
# # weighted_mosaic_data["EVI"] = 2.5 * ((weighted_mosaic_data.B08 - weighted_mosaic_data.B04) / (weighted_mosaic_data.B08 + 6 * weighted_mosaic_data.B04 - 7.5 * weighted_mosaic_data.B02 + 1))
# # weighted_mosaic_data["SAVI"] = ((weighted_mosaic_data.B08 - weighted_mosaic_data.B04) * 1.5) / (weighted_mosaic_data.B08 + weighted_mosaic_data.B04 + 0.5)
# # weighted_mosaic_data["MSAVI"] = (2 * weighted_mosaic_data.B08 + 1 - ((2 * weighted_mosaic_data.B08 + 1) ** 2 - 8 * (weighted_mosaic_data.B08 - weighted_mosaic_data.B04)) ** 0.5) / 2
# # weighted_mosaic_data["GNDVI"] = (weighted_mosaic_data.B08 - weighted_mosaic_data.B03) / (weighted_mosaic_data.B08 + weighted_mosaic_data.B03)
# # weighted_mosaic_data["IBI"] = (2 * weighted_mosaic_data.B11 - (weighted_mosaic_data.B08 + weighted_mosaic_data.B04)) / (weighted_mosaic_data.B11 + weighted_mosaic_data.B08 + weighted_mosaic_data.B04)
# # weighted_mosaic_data["UI"] = (weighted_mosaic_data.B08 + weighted_mosaic_data.B11) / (weighted_mosaic_data.B02 + weighted_mosaic_data.B03)
# # weighted_mosaic_data["NDBSI"] = (weighted_mosaic_data.B03 + weighted_mosaic_data.B11) / (weighted_mosaic_data.B08 + weighted_mosaic_data.B02)
# # weighted_mosaic_data["NDMI"] = (weighted_mosaic_data.B08 - weighted_mosaic_data.B11) / (weighted_mosaic_data.B08 + weighted_mosaic_data.B11)
# # weighted_mosaic_data["NDSI"] = (weighted_mosaic_data.B03 - weighted_mosaic_data.B11) / (weighted_mosaic_data.B03 + weighted_mosaic_data.B11)
# # weighted_mosaic_data["BI"] = (weighted_mosaic_data.B03 + weighted_mosaic_data.B04 + weighted_mosaic_data.B08 + weighted_mosaic_data.B11) / 4

# # # Add new features
# # weighted_mosaic_data["REP"] = 705 + 35 * (((weighted_mosaic_data.B05 + weighted_mosaic_data.B07) / 2) - weighted_mosaic_data.B06) / (weighted_mosaic_data.B07 - weighted_mosaic_data.B05)
# # weighted_mosaic_data["NGRDI"] = (weighted_mosaic_data.B03 - weighted_mosaic_data.B04) / (weighted_mosaic_data.B03 + weighted_mosaic_data.B04)
# # weighted_mosaic_data["MNDWI"] = (weighted_mosaic_data.B03 - weighted_mosaic_data.B11) / (weighted_mosaic_data.B03 + weighted_mosaic_data.B11)
# # weighted_mosaic_data["NDWI_Variant"] = (weighted_mosaic_data.B03 - weighted_mosaic_data.B8A) / (weighted_mosaic_data.B03 + weighted_mosaic_data.B8A)
# # weighted_mosaic_data["BSI"] = (weighted_mosaic_data.B11 + weighted_mosaic_data.B04 - weighted_mosaic_data.B08 - weighted_mosaic_data.B02) / (weighted_mosaic_data.B11 + weighted_mosaic_data.B04 + weighted_mosaic_data.B08 + weighted_mosaic_data.B02)
# # weighted_mosaic_data["SBI"] = ((weighted_mosaic_data.B03 ** 2 + weighted_mosaic_data.B04 ** 2 + weighted_mosaic_data.B11 ** 2) ** 0.5)
# # weighted_mosaic_data["Albedo"] = (weighted_mosaic_data.B02 + weighted_mosaic_data.B03 + weighted_mosaic_data.B04 + weighted_mosaic_data.B8A + weighted_mosaic_data.B11) / 5
# # weighted_mosaic_data["Vegetation_Ratio"] = ((weighted_mosaic_data["SCL"] == 4).sum(dim=["latitude", "longitude"]) / weighted_mosaic_data["SCL"].size) * 100
# # weighted_mosaic_data["Bare_Soil_Ratio"] = ((weighted_mosaic_data["SCL"] == 5).sum(dim=["latitude", "longitude"]) / weighted_mosaic_data["SCL"].size) * 100
# # weighted_mosaic_data["Water_Ratio"] = ((weighted_mosaic_data["SCL"] == 6).sum(dim=["latitude", "longitude"]) / weighted_mosaic_data["SCL"].size) * 100

# # # Simplified binary masks
# # weighted_mosaic_data["Urban_Ratio"] = ((weighted_mosaic_data["SCL"] == 5).sum(dim=["latitude", "longitude"]) / weighted_mosaic_data["SCL"].size) * 100
# # weighted_mosaic_data["Cloud_Shadow_Ratio"] = ((weighted_mosaic_data["SCL"] == 3).sum(dim=["latitude", "longitude"]) / weighted_mosaic_data["SCL"].size) * 100
# # Save weighted mosaic to GeoTIFF
# weighted_mosaic_tiff_path = "Sentinel2_WeightedMosaic_(nbs).tiff"
# weighted_mosaic_data.rio.to_raster(weighted_mosaic_tiff_path, compress="lzw")
# print(f"Weighted mosaic GeoTIFF with all features saved at {weighted_mosaic_tiff_path}")

# # save_resampled_weighted_mosaics(weighted_mosaic_data, output_prefix='/kaggle/working/Sentinel2_Mosaic_30m_no_base_sampling')

In [30]:
# resampled_30m_ds = resample_to_30m(weighted_mosaic_data, method="bilinear")
# save_mosaic_to_tiff(resampled_30m_ds, "/kaggle/working/sentinel_30m_billinear(nbs).tiff", bounds)

# # resampled_30m_ds = resample_to_30m(weighted_mosaic_data, method="cubic")
# # save_mosaic_to_tiff(resampled_30m_ds, "/kaggle/working/sentinel_30m_cubic(nbs).tiff", bounds)

In [31]:
# evi_median = (2.5 * ((median.B08 - median.B04) / (median.B08 + (6 * median.B04) - (7.5 * median.B02) + 1)))
# L = 0.5  # Soil brightness correction factor
# savi_median = (((median.B08 - median.B04) / (median.B08 + median.B04 + L)) * (1 + L))
# msavi_median = (((2 * median.B08) + 1) - (((2 * median.B08 + 1) ** 2) - (8 * (median.B08 - median.B04))) ** 0.5) / 2
# gndvi_median = ((median.B08 - median.B03) / (median.B08 + median.B03))

# ibi_median = ((ndbi_median - (ndvi_median + ndwi_median)) / (ndbi_median + (ndvi_median + ndwi_median)))

# ui_median = ((median.B11 - median.B08) / (median.B11 + median.B08))
# ndbsi_median = ((ndbi_median + (1 - ndvi_median)) / 2)
# ndmi_median = ((median.B08 - median.B11) / (median.B08 + median.B11))
# ndsi_median = ((median.B03 - median.B11) / (median.B03 + median.B11))
# bi_median = (((median.B04 ** 2) + (median.B03 ** 2) + (median.B02 ** 2)) ** 0.5) / 3


In [32]:
# def plot_all_indices(indices_dict, vmin_vmax_dict, cmap_dict, title_dict):
#     """
#     Plots all indices in subplots using a grid layout.

#     Parameters:
#     indices_dict: dict
#         A dictionary where keys are index names (e.g., 'NDVI') and values are the calculated xarray indices.
#     vmin_vmax_dict: dict
#         A dictionary where keys are index names and values are tuples of (vmin, vmax) for color scale.
#     cmap_dict: dict
#         A dictionary where keys are index names and values are the colormap to use for each index.
#     title_dict: dict
#         A dictionary where keys are index names and values are the title for each plot.
#     """
#     num_indices = len(indices_dict)
#     ncols = 3  # Number of columns in the grid
#     nrows = (num_indices + ncols - 1) // ncols  # Calculate rows needed for the grid

#     fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 5 * nrows))
#     axes = axes.flatten()  # Flatten axes for easier iteration

#     for i, (index_name, index_data) in enumerate(indices_dict.items()):
#         ax = axes[i]
#         index_data.plot.imshow(ax=ax, vmin=vmin_vmax_dict[index_name][0], 
#                                vmax=vmin_vmax_dict[index_name][1], cmap=cmap_dict[index_name])
#         ax.set_title(title_dict[index_name])
#         ax.axis('off')

#     # Hide any unused subplots
#     for j in range(i + 1, len(axes)):
#         fig.delaxes(axes[j])

#     plt.tight_layout()
#     plt.show()

# # Define all indices and their properties
# indices_dict = {
#     "NDVI": ndvi_median,
#     "NDWI": ndwi_median,
#     "NDBI": ndbi_median,
#     "EVI": evi_median,
#     "SAVI": savi_median,
#     "MSAVI": msavi_median,
#     "GNDVI": gndvi_median,
#     "IBI": ibi_median,
#     "UI": ui_median,
#     "NDBSI": ndbsi_median,
#     "NDMI": ndmi_median,
#     "NDSI": ndsi_median,
#     "BI": bi_median,
# }

# # Define vmin and vmax for each index
# vmin_vmax_dict = {
#     "NDVI": (0.0, 1.0),
#     "NDWI": (-0.3, 0.3),
#     "NDBI": (-0.1, 0.1),
#     "EVI": (-1.0, 1.0),
#     "SAVI": (0.0, 1.0),
#     "MSAVI": (0.0, 1.0),
#     "GNDVI": (0.0, 1.0),
#     "IBI": (-1.0, 1.0),
#     "UI": (-1.0, 1.0),
#     "NDBSI": (0.0, 1.0),
#     "NDMI": (-1.0, 1.0),
#     "NDSI": (-1.0, 1.0),
#     "BI": (0.0, 1.0),
# }

# # Define colormaps for each index
# cmap_dict = {
#     "NDVI": "RdYlGn",
#     "NDWI": "RdBu",
#     "NDBI": "jet",
#     "EVI": "RdYlGn",
#     "SAVI": "RdYlGn",
#     "MSAVI": "RdYlGn",
#     "GNDVI": "RdYlGn",
#     "IBI": "coolwarm",
#     "UI": "coolwarm",
#     "NDBSI": "YlOrRd",
#     "NDMI": "PiYG",
#     "NDSI": "BrBG",
#     "BI": "cividis",
# }

# # Define titles for each index
# title_dict = {
#     "NDVI": "Normalized Difference Vegetation Index (NDVI)",
#     "NDWI": "Normalized Difference Water Index (NDWI)",
#     "NDBI": "Normalized Difference Built-Up Index (NDBI)",
#     "EVI": "Enhanced Vegetation Index (EVI)",
#     "SAVI": "Soil-Adjusted Vegetation Index (SAVI)",
#     "MSAVI": "Modified Soil-Adjusted Vegetation Index (MSAVI)",
#     "GNDVI": "Green Normalized Difference Vegetation Index (GNDVI)",
#     "IBI": "Index-Based Built-Up Index (IBI)",
#     "UI": "Urban Index (UI)",
#     "NDBSI": "Normalized Difference Bare Soil Index (NDBSI)",
#     "NDMI": "Normalized Difference Moisture Index (NDMI)",
#     "NDSI": "Normalized Difference Snow Index (NDSI)",
#     "BI": "Brightness Index (BI)",
# }

# # Call the function to plot all indices
# # plot_all_indices(indices_dict, vmin_vmax_dict, cmap_dict, title_dict)

In [33]:
# mean_data = data.mean(dim='time').compute()

# # bands_only_path_mean = "Sentinel2_Mean_Only_Bands.tiff"
# # mean_data.rio.to_raster(bands_only_path_mean, compress="lzw")
# # print(f"Mean GeoTIFF with only bands saved at {bands_only_path_mean}")

# # Add all derived indices directly
# mean_data["NDVI"] = (mean_data.B08 - mean_data.B04) / (mean_data.B08 + mean_data.B04)
# mean_data["NDBI"] = (mean_data.B11 - mean_data.B08) / (mean_data.B11 + mean_data.B08)
# mean_data["NDWI"] = (mean_data.B03 - mean_data.B08) / (mean_data.B03 + mean_data.B08)
# mean_data["EVI"] = 2.5 * ((mean_data.B08 - mean_data.B04) / (mean_data.B08 + 6 * mean_data.B04 - 7.5 * mean_data.B02 + 1))
# mean_data["SAVI"] = ((mean_data.B08 - mean_data.B04) * 1.5) / (mean_data.B08 + mean_data.B04 + 0.5)
# mean_data["MSAVI"] = (2 * mean_data.B08 + 1 - ((2 * mean_data.B08 + 1) ** 2 - 8 * (mean_data.B08 - mean_data.B04)) ** 0.5) / 2
# mean_data["GNDVI"] = (mean_data.B08 - mean_data.B03) / (mean_data.B08 + mean_data.B03)
# mean_data["IBI"] = (2 * mean_data.B11 - (mean_data.B08 + mean_data.B04)) / (mean_data.B11 + mean_data.B08 + mean_data.B04)
# mean_data["UI"] = (mean_data.B08 + mean_data.B11) / (mean_data.B02 + mean_data.B03)
# mean_data["NDBSI"] = (mean_data.B03 + mean_data.B11) / (mean_data.B08 + mean_data.B02)
# mean_data["NDMI"] = (mean_data.B08 - mean_data.B11) / (mean_data.B08 + mean_data.B11)
# mean_data["NDSI"] = (mean_data.B03 - mean_data.B11) / (mean_data.B03 + mean_data.B11)
# mean_data["BI"] = (mean_data.B03 + mean_data.B04 + mean_data.B08 + mean_data.B11) / 4

# mean_data["REP"] = 705 + 35 * (((mean_data.B05 + mean_data.B07) / 2) - mean_data.B06) / (mean_data.B07 - mean_data.B05)
# mean_data["NGRDI"] = (mean_data.B03 - mean_data.B04) / (mean_data.B03 + mean_data.B04)
# mean_data["MNDWI"] = (mean_data.B03 - mean_data.B11) / (mean_data.B03 + mean_data.B11)
# mean_data["NDWI_Variant"] = (mean_data.B03 - mean_data.B8A) / (mean_data.B03 + mean_data.B8A)
# mean_data["BSI"] = (mean_data.B11 + mean_data.B04 - mean_data.B08 - mean_data.B02) / (mean_data.B11 + mean_data.B04 + mean_data.B08 + mean_data.B02)
# mean_data["SBI"] = ((mean_data.B03 ** 2 + mean_data.B04 ** 2 + mean_data.B11 ** 2) ** 0.5)
# mean_data["Albedo"] = (mean_data.B02 + mean_data.B03 + mean_data.B04 + mean_data.B8A + mean_data.B11) / 5

# mean_data["Vegetation_Ratio"] = ((mean_data["SCL"] == 4).sum(dim=["latitude", "longitude"]) / mean_data["SCL"].size) * 100
# mean_data["Bare_Soil_Ratio"] = ((mean_data["SCL"] == 5).sum(dim=["latitude", "longitude"]) / mean_data["SCL"].size) * 100
# mean_data["Water_Ratio"] = ((mean_data["SCL"] == 6).sum(dim=["latitude", "longitude"]) / mean_data["SCL"].size) * 100

# # Simplified binary masks
# mean_data["Urban_Ratio"] = ((mean_data["SCL"] == 5).sum(dim=["latitude", "longitude"]) / mean_data["SCL"].size) * 100
# mean_data["Cloud_Shadow_Ratio"] = ((mean_data["SCL"] == 3).sum(dim=["latitude", "longitude"]) / mean_data["SCL"].size) * 100
# # Save median composite to a GeoTIFF file
# mean_tiff_path = "Sentinel2_Mean_All_Bands_and_Indices.tiff"
# mean_data.rio.to_raster(mean_tiff_path, compress="lzw")
# print(f"Mean GeoTIFF saved at {mean_tiff_path}")


In [34]:
# fig, ax = plt.subplots(figsize=(6,6))
# mean_data[["B04", "B03", "B02"]].to_array().plot.imshow(robust=True, ax=ax, vmin=0, vmax=2500)
# ax.set_title("RGB Mean Composite")
# ax.axis('off')
# plt.show()

In [35]:
# # Save Median Composite GeoTIFF
# # Calculate the median composite
# median_data = data.median(dim="time").compute()

# # Save version with only spectral bands
# # bands_only_path_median = "Sentinel2_Median_Only_Bands.tiff"
# # median_data.rio.to_raster(bands_only_path_median, compress="lzw")
# # print(f"Median GeoTIFF with only bands saved at {bands_only_path_median}")

# # Add all derived indices directly
# median_data["NDVI"] = (median_data.B08 - median_data.B04) / (median_data.B08 + median_data.B04)
# median_data["NDBI"] = (median_data.B11 - median_data.B08) / (median_data.B11 + median_data.B08)
# median_data["NDWI"] = (median_data.B03 - median_data.B08) / (median_data.B03 + median_data.B08)
# median_data["EVI"] = 2.5 * ((median_data.B08 - median_data.B04) / (median_data.B08 + 6 * median_data.B04 - 7.5 * median_data.B02 + 1))
# median_data["SAVI"] = ((median_data.B08 - median_data.B04) * 1.5) / (median_data.B08 + median_data.B04 + 0.5)
# median_data["MSAVI"] = (2 * median_data.B08 + 1 - ((2 * median_data.B08 + 1) ** 2 - 8 * (median_data.B08 - median_data.B04)) ** 0.5) / 2
# median_data["GNDVI"] = (median_data.B08 - median_data.B03) / (median_data.B08 + median_data.B03)
# median_data["IBI"] = (2 * median_data.B11 - (median_data.B08 + median_data.B04)) / (median_data.B11 + median_data.B08 + median_data.B04)
# median_data["UI"] = (median_data.B08 + median_data.B11) / (median_data.B02 + median_data.B03)
# median_data["NDBSI"] = (median_data.B03 + median_data.B11) / (median_data.B08 + median_data.B02)
# median_data["NDMI"] = (median_data.B08 - median_data.B11) / (median_data.B08 + median_data.B11)
# median_data["NDSI"] = (median_data.B03 - median_data.B11) / (median_data.B03 + median_data.B11)
# median_data["BI"] = (median_data.B03 + median_data.B04 + median_data.B08 + median_data.B11) / 4

# median_data["REP"] = 705 + 35 * (((median_data.B05 + median_data.B07) / 2) - median_data.B06) / (median_data.B07 - median_data.B05)
# median_data["NGRDI"] = (median_data.B03 - median_data.B04) / (median_data.B03 + median_data.B04)
# median_data["MNDWI"] = (median_data.B03 - median_data.B11) / (median_data.B03 + median_data.B11)
# median_data["NDWI_Variant"] = (median_data.B03 - median_data.B8A) / (median_data.B03 + median_data.B8A)
# median_data["BSI"] = (median_data.B11 + median_data.B04 - median_data.B08 - median_data.B02) / (median_data.B11 + median_data.B04 + median_data.B08 + median_data.B02)
# median_data["SBI"] = ((median_data.B03 ** 2 + median_data.B04 ** 2 + median_data.B11 ** 2) ** 0.5)
# median_data["Albedo"] = (median_data.B02 + median_data.B03 + median_data.B04 + median_data.B8A + median_data.B11) / 5

# median_data["Vegetation_Ratio"] = ((median_data["SCL"] == 4).sum(dim=["latitude", "longitude"]) / median_data["SCL"].size) * 100
# median_data["Bare_Soil_Ratio"] = ((median_data["SCL"] == 5).sum(dim=["latitude", "longitude"]) / median_data["SCL"].size) * 100
# median_data["Water_Ratio"] = ((median_data["SCL"] == 6).sum(dim=["latitude", "longitude"]) / median_data["SCL"].size) * 100

# # Simplified binary masks
# median_data["Urban_Ratio"] = ((median_data["SCL"] == 5).sum(dim=["latitude", "longitude"]) / median_data["SCL"].size) * 100
# median_data["Cloud_Shadow_Ratio"] = ((median_data["SCL"] == 3).sum(dim=["latitude", "longitude"]) / median_data["SCL"].size) * 100
# # Save median composite to a GeoTIFF file
# median_tiff_path = "Sentinel2_Median_All_Bands_and_Indices.tiff"
# median_data.rio.to_raster(median_tiff_path, compress="lzw")
# print(f"Median GeoTIFF saved at {median_tiff_path}")


In [36]:
# fig, ax = plt.subplots(figsize=(6,6))
# median_data[["B04", "B03", "B02"]].to_array().plot.imshow(robust=True, ax=ax, vmin=0, vmax=2500)
# ax.set_title("RGB Median Composite")
# ax.axis('off')
# plt.show()

In [37]:
# # Save Single-Date GeoTIFF
# # Select the specific date (2021-07-24)
# single_date_index = 6  # Ensure this index corresponds to the correct date
# single_date_data = data.isel(time=single_date_index)

# # Save version with only spectral bands
# # bands_only_path_single = "Sentinel2_SingleDate_Only_Bands.tiff"
# # single_date_data.rio.to_raster(bands_only_path_single, compress="lzw")
# # print(f"Single-date GeoTIFF with only bands saved at {bands_only_path_single}")

# # Add all derived indices directly
# single_date_data["NDVI"] = (single_date_data.B08 - single_date_data.B04) / (single_date_data.B08 + single_date_data.B04)
# single_date_data["NDBI"] = (single_date_data.B11 - single_date_data.B08) / (single_date_data.B11 + single_date_data.B08)
# single_date_data["NDWI"] = (single_date_data.B03 - single_date_data.B08) / (single_date_data.B03 + single_date_data.B08)
# single_date_data["EVI"] = 2.5 * ((single_date_data.B08 - single_date_data.B04) / (single_date_data.B08 + 6 * single_date_data.B04 - 7.5 * single_date_data.B02 + 1))
# single_date_data["SAVI"] = ((single_date_data.B08 - single_date_data.B04) * 1.5) / (single_date_data.B08 + single_date_data.B04 + 0.5)
# single_date_data["MSAVI"] = (2 * single_date_data.B08 + 1 - ((2 * single_date_data.B08 + 1) ** 2 - 8 * (single_date_data.B08 - single_date_data.B04)) ** 0.5) / 2
# single_date_data["GNDVI"] = (single_date_data.B08 - single_date_data.B03) / (single_date_data.B08 + single_date_data.B03)
# single_date_data["IBI"] = (2 * single_date_data.B11 - (single_date_data.B08 + single_date_data.B04)) / (single_date_data.B11 + single_date_data.B08 + single_date_data.B04)
# single_date_data["UI"] = (single_date_data.B08 + single_date_data.B11) / (single_date_data.B02 + single_date_data.B03)
# single_date_data["NDBSI"] = (single_date_data.B03 + single_date_data.B11) / (single_date_data.B08 + single_date_data.B02)
# single_date_data["NDMI"] = (single_date_data.B08 - single_date_data.B11) / (single_date_data.B08 + single_date_data.B11)
# single_date_data["NDSI"] = (single_date_data.B03 - single_date_data.B11) / (single_date_data.B03 + single_date_data.B11)
# single_date_data["BI"] = (single_date_data.B03 + single_date_data.B04 + single_date_data.B08 + single_date_data.B11) / 4
# single_date_data["REP"] = 705 + 35 * (((single_date_data.B05 + single_date_data.B07) / 2) - single_date_data.B06) / (single_date_data.B07 - single_date_data.B05)
# single_date_data["NGRDI"] = (single_date_data.B03 - single_date_data.B04) / (single_date_data.B03 + single_date_data.B04)
# single_date_data["MNDWI"] = (single_date_data.B03 - single_date_data.B11) / (single_date_data.B03 + single_date_data.B11)
# single_date_data["NDWI_Variant"] = (single_date_data.B03 - single_date_data.B8A) / (single_date_data.B03 + single_date_data.B8A)
# single_date_data["BSI"] = (single_date_data.B11 + single_date_data.B04 - single_date_data.B08 - single_date_data.B02) / (single_date_data.B11 + single_date_data.B04 + single_date_data.B08 + single_date_data.B02)
# single_date_data["SBI"] = ((single_date_data.B03 ** 2 + single_date_data.B04 ** 2 + single_date_data.B11 ** 2) ** 0.5)
# single_date_data["Albedo"] = (single_date_data.B02 + single_date_data.B03 + single_date_data.B04 + single_date_data.B8A + single_date_data.B11) / 5
# single_date_data["Vegetation_Ratio"] = ((single_date_data["SCL"] == 4).sum(dim=["latitude", "longitude"]) / single_date_data["SCL"].size) * 100
# single_date_data["Bare_Soil_Ratio"] = ((single_date_data["SCL"] == 5).sum(dim=["latitude", "longitude"]) / single_date_data["SCL"].size) * 100
# single_date_data["Water_Ratio"] = ((single_date_data["SCL"] == 6).sum(dim=["latitude", "longitude"]) / single_date_data["SCL"].size) * 100

# # Simplified binary masks
# single_date_data["Urban_Ratio"] = ((single_date_data["SCL"] == 5).sum(dim=["latitude", "longitude"]) / single_date_data["SCL"].size) * 100
# single_date_data["Cloud_Shadow_Ratio"] = ((single_date_data["SCL"] == 3).sum(dim=["latitude", "longitude"]) / single_date_data["SCL"].size) * 100
# # Save single-date data to a GeoTIFF file
# single_date_tiff_path = "Sentinel2_SingleDate_All_Bands_and_Indices.tiff"
# single_date_data.rio.to_raster(single_date_tiff_path, compress="lzw")
# print(f"Single-date GeoTIFF saved at {single_date_tiff_path}")


In [38]:
# fig, ax = plt.subplots(figsize=(6,6))
# single_date_data[["B04", "B03", "B02"]].to_array().plot.imshow(robust=True, ax=ax, vmin=0, vmax=2500)
# ax.set_title("RGB Single date(24-July-2024) Composite")
# ax.axis('off')
# plt.show()

In [39]:
# from tqdm import tqdm

# # Select the specific date (2021-07-24)
# single_date_masked_index = 6  # Ensure this index corresponds to the correct date
# single_date_masked_data = data.isel(time=single_date_masked_index)

# # Apply Cloud Masking (using SCL band)
# cloud_mask = ~((single_date_masked_data["SCL"] == 8) | (single_date_masked_data["SCL"] == 9) | (single_date_masked_data["SCL"] == 10) | (single_date_masked_data["SCL"] == 11))
# for band in single_date_masked_data.data_vars:
#     if band != "SCL":  # Skip the SCL layer
#         single_date_masked_data[band] = single_date_masked_data[band].where(cloud_mask)

# # Add all derived indices directly
# single_date_masked_data["NDVI"] = (single_date_masked_data.B08 - single_date_masked_data.B04) / (single_date_masked_data.B08 + single_date_masked_data.B04)
# single_date_masked_data["NDBI"] = (single_date_masked_data.B11 - single_date_masked_data.B08) / (single_date_masked_data.B11 + single_date_masked_data.B08)
# single_date_masked_data["NDWI"] = (single_date_masked_data.B03 - single_date_masked_data.B08) / (single_date_masked_data.B03 + single_date_masked_data.B08)
# single_date_masked_data["EVI"] = 2.5 * ((single_date_masked_data.B08 - single_date_masked_data.B04) / (single_date_masked_data.B08 + 6 * single_date_masked_data.B04 - 7.5 * single_date_masked_data.B02 + 1))
# single_date_masked_data["SAVI"] = ((single_date_masked_data.B08 - single_date_masked_data.B04) * 1.5) / (single_date_masked_data.B08 + single_date_masked_data.B04 + 0.5)
# single_date_masked_data["MSAVI"] = (2 * single_date_masked_data.B08 + 1 - ((2 * single_date_masked_data.B08 + 1) ** 2 - 8 * (single_date_masked_data.B08 - single_date_masked_data.B04)) ** 0.5) / 2
# single_date_masked_data["GNDVI"] = (single_date_masked_data.B08 - single_date_masked_data.B03) / (single_date_masked_data.B08 + single_date_masked_data.B03)
# single_date_masked_data["IBI"] = (2 * single_date_masked_data.B11 - (single_date_masked_data.B08 + single_date_masked_data.B04)) / (single_date_masked_data.B11 + single_date_masked_data.B08 + single_date_masked_data.B04)
# single_date_masked_data["UI"] = (single_date_masked_data.B08 + single_date_masked_data.B11) / (single_date_masked_data.B02 + single_date_masked_data.B03)
# single_date_masked_data["NDBSI"] = (single_date_masked_data.B03 + single_date_masked_data.B11) / (single_date_masked_data.B08 + single_date_masked_data.B02)
# single_date_masked_data["NDMI"] = (single_date_masked_data.B08 - single_date_masked_data.B11) / (single_date_masked_data.B08 + single_date_masked_data.B11)
# single_date_masked_data["NDSI"] = (single_date_masked_data.B03 - single_date_masked_data.B11) / (single_date_masked_data.B03 + single_date_masked_data.B11)
# single_date_masked_data["BI"] = (single_date_masked_data.B03 + single_date_masked_data.B04 + single_date_masked_data.B08 + single_date_masked_data.B11) / 4

# single_date_masked_data["REP"] = 705 + 35 * (((single_date_masked_data.B05 + single_date_masked_data.B07) / 2) - single_date_masked_data.B06) / (single_date_masked_data.B07 - single_date_masked_data.B05)
# single_date_masked_data["NGRDI"] = (single_date_masked_data.B03 - single_date_masked_data.B04) / (single_date_masked_data.B03 + single_date_masked_data.B04)
# single_date_masked_data["MNDWI"] = (single_date_masked_data.B03 - single_date_masked_data.B11) / (single_date_masked_data.B03 + single_date_masked_data.B11)
# single_date_masked_data["NDWI_Variant"] = (single_date_masked_data.B03 - single_date_masked_data.B8A) / (single_date_masked_data.B03 + single_date_masked_data.B8A)
# single_date_masked_data["BSI"] = (single_date_masked_data.B11 + single_date_masked_data.B04 - single_date_masked_data.B08 - single_date_masked_data.B02) / (single_date_masked_data.B11 + single_date_masked_data.B04 + single_date_masked_data.B08 + single_date_masked_data.B02)
# single_date_masked_data["SBI"] = ((single_date_masked_data.B03 ** 2 + single_date_masked_data.B04 ** 2 + single_date_masked_data.B11 ** 2) ** 0.5)
# single_date_masked_data["Albedo"] = (single_date_masked_data.B02 + single_date_masked_data.B03 + single_date_masked_data.B04 + single_date_masked_data.B8A + single_date_masked_data.B11) / 5
# single_date_masked_data["Cloud_Percentage"] = (cloud_mask.sum() / single_date_masked_data["SCL"].size) * 100
# single_date_masked_data["Vegetation_Ratio"] = ((single_date_masked_data["SCL"] == 4).sum(dim=["latitude", "longitude"]) / single_date_masked_data["SCL"].size) * 100
# single_date_masked_data["Bare_Soil_Ratio"] = ((single_date_masked_data["SCL"] == 5).sum(dim=["latitude", "longitude"]) / single_date_masked_data["SCL"].size) * 100
# single_date_masked_data["Water_Ratio"] = ((single_date_masked_data["SCL"] == 6).sum(dim=["latitude", "longitude"]) / single_date_masked_data["SCL"].size) * 100

# # Simplified binary masks
# single_date_masked_data["Urban_Ratio"] = ((single_date_masked_data["SCL"] == 5).sum(dim=["latitude", "longitude"]) / single_date_masked_data["SCL"].size) * 100
# single_date_masked_data["Cloud_Shadow_Ratio"] = ((single_date_masked_data["SCL"] == 3).sum(dim=["latitude", "longitude"]) / single_date_masked_data["SCL"].size) * 100

# # Save single-date data to a GeoTIFF file
# single_date_masked_tiff_path = "Sentinel2_SingleDate_CloudMasked_All_Bands_and_Indices.tiff"
# single_date_masked_data.rio.to_raster(single_date_masked_tiff_path, compress="lzw")
# print(f"Single-date GeoTIFF with cloud masking saved at {single_date_masked_tiff_path}")


In [40]:
# # Plot the SCL band to verify cloud classification
# single_date_masked_data["SCL"].plot(cmap="viridis", robust=True)
# plt.title("SCL Band for Cloud Classification")
# plt.show()

# # Ensure Dask arrays are computed first
# nan_count = single_date_masked_data["B04"].isnull().sum().compute()  # Total NaN count
# total_count = single_date_masked_data["B04"].size  # Total number of elements in the array

# # Calculate NaN percentage
# nan_percentage = (nan_count / total_count) * 100
# print(f"Percentage of NaN values in B04 (masked): {nan_percentage:.2f}%")



In [41]:
# fig, ax = plt.subplots(figsize=(6, 6))
# rgb = single_date_masked_data[["B04", "B03", "B02"]].to_array()

# rgb.plot.imshow(robust=True, ax=ax, vmin=0, vmax=2500, cmap="viridis", add_colorbar=True)
# ax.set_title("RGB Single Date with Cloud Masking (24-July-2024)")
# ax.axis('off')
# plt.show()